# Backtesting the 'buying-the-dip' Trading Strategy

## Libraries and settings

In [ ]:
# Libraries
import os
import numpy as np
import pandas as pd
import yfinance as yf
from datetime import datetime
import matplotlib.pyplot as plt

# Settings
import warnings
warnings.filterwarnings("ignore")

# Current working directory
print(f'Current working directory: {os.getcwd()}')

## Historical Data iShares Core MSCI World UCITS ETF

In [ ]:
# Get today's date
today = datetime.today().strftime('%Y-%m-%d')

# Load historical data iShares Core MSCI World UCITS ETF
ticker = 'IWDA.AS'
df = yf.download(ticker, 
                 start='2010-01-01', 
                 end=today, 
                 interval='1mo',
                 multi_level_index=False,
                 progress=False)


# Check currency
ticker = yf.Ticker("IWDA.AS")
print(f"The currency used by yfinance is: {ticker.info['currency']}")

# Monthly return
df['Monthly_Return'] = df['Close'].pct_change()
df

## Backtesting buying-the-dip versus fixed monthly investment Strategy

In [ ]:
# Strategy 1: Invest a fixed amount every month
fixed_investment = 1000
df['Fixed Investment'] = fixed_investment
df['Fixed Shares'] = df['Fixed Investment'] / df['Close']
df['Fixed Total Shares'] = df['Fixed Shares'].cumsum()
df['Fixed Portfolio Value'] = df['Fixed Total Shares'] * df['Close']

# Strategy 2: Invest cumulative monthly savings after a drop of X%
threshold = 0.05
monthly_savings = 1000
df['Trigger Investment'] = 0.0
df['Trigger Shares'] = 0.0 
savings = 0.0

# Ensure that the index is a simple integer
df = df.reset_index()

for i in range(1, len(df)):
    if threshold == 0.00:
        # Invest fixed savings every month if the threshold is 0
        df.loc[i, 'Trigger Investment'] = monthly_savings
    else:
        # Accumulate savings if the threshold is > 0
        savings += monthly_savings
        # Invest the accumulated savings if the price drops by at least threshold
        if df.loc[i, 'Close'] <= df.loc[i-1, 'Close'] * (1 - threshold):
            df.loc[i, 'Trigger Investment'] = savings
            savings = 0

    # Calculate the number of shares bought
    df.loc[i, 'Trigger Shares'] = (
        df.loc[i, 'Trigger Investment'] / df.loc[i, 'Close']
    )

# Calculate the total number of shares and the portfolio value
df['Trigger Total Shares'] = df['Trigger Shares'].cumsum()
df['Trigger Portfolio Value'] = df['Trigger Total Shares'] * df['Close']

# Output the final portfolio values
print(f"Final portfolio value with fixed monthly investment: {df['Fixed Portfolio Value'].iloc[-1]:.2f} EUR")
print(f"Final portfolio value with investment after a drop: {df['Trigger Portfolio Value'].iloc[-1]:.2f} EUR")

# Difference in portfolio values
diff = df['Trigger Portfolio Value'].iloc[-1] - df['Fixed Portfolio Value'].iloc[-1]
print(f"\nDifference in portfolio values: {diff:.2f} EUR")

## Plot the results

In [ ]:
# Plot the results
plt.figure(figsize=(8, 5))
plt.plot(df['Date'], df['Fixed Portfolio Value'], label='Fixed monthly investment')
plt.plot(df['Date'], df['Trigger Portfolio Value'], label='Investment after drop')
plt.title('Backtesting of Investment Strategies')
plt.xlabel('Date')
plt.ylabel('Portfolio value')
plt.legend()
plt.grid(True)
plt.show()

### Jupyter notebook --footer info-- (please always provide this at the end of each notebook)

In [ ]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')